[DuckDB 0.9.0](https://duckdb.org/2023/09/26/announcing-duckdb-090.html)

```
$ conda env list
$ conda create -n duck090
$ conda activate duck090
$ pip install duckdb==0.9.0
$ conda deactivate 
```

In [1]:
from pathlib import Path 

In [2]:
import duckdb 

In [3]:
duckdb.__version__

'0.9.0'

In [4]:
df = duckdb.sql("FROM 'https://raw.githubusercontent.com/duckdb/duckdb/main/data/json/example_n.ndjson'")

In [5]:
df

┌───────┬────────────────────────────┐
│  id   │            name            │
│ int64 │          varchar           │
├───────┼────────────────────────────┤
│     1 │ O Brother, Where Art Thou? │
│     2 │ Home for the Holidays      │
│     3 │ The Firm                   │
│     4 │ Broadcast News             │
│     5 │ Raising Arizona            │
└───────┴────────────────────────────┘

In [6]:
df.shape 

(5, 2)

In [7]:
!cd 

C:\Users\p2p2l\projects\wgong\py4kids\lesson-14.6-polars\polars-cookbook\cookbook


In [8]:
file_path = "../data/NYC-parking-tickets/Parking_Violations_Issued_-_Fiscal_Year_2014.csv"

In [15]:
df = duckdb.read_csv(file_path, parallel=True, all_varchar=True)

In [16]:
type(df)

duckdb.duckdb.DuckDBPyRelation

In [17]:
df.shape

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

(9100278, 51)

In [18]:
df.columns

['Summons Number',
 'Plate ID',
 'Registration State',
 'Plate Type',
 'Issue Date',
 'Violation Code',
 'Vehicle Body Type',
 'Vehicle Make',
 'Issuing Agency',
 'Street Code1',
 'Street Code2',
 'Street Code3',
 'Vehicle Expiration Date',
 'Violation Location',
 'Violation Precinct',
 'Issuer Precinct',
 'Issuer Code',
 'Issuer Command',
 'Issuer Squad',
 'Violation Time',
 'Time First Observed',
 'Violation County',
 'Violation In Front Of Or Opposite',
 'House Number',
 'Street Name',
 'Intersecting Street',
 'Date First Observed',
 'Law Section',
 'Sub Division',
 'Violation Legal Code',
 'Days Parking In Effect',
 'From Hours In Effect',
 'To Hours In Effect',
 'Vehicle Color',
 'Unregistered Vehicle?',
 'Vehicle Year',
 'Meter Number',
 'Feet From Curb',
 'Violation Post Code',
 'Violation Description',
 'No Standing or Stopping Violation',
 'Hydrant Violation',
 'Double Parking Violation',
 'Latitude',
 'Longitude',
 'Community Board',
 'Community Council',
 'Census Tract',
 'B

## Create Table 

In [28]:
%%timeit
# 97.6 ms
tab_name = "parking"
sql_stmt = f"""
    create table IF NOT EXISTS {tab_name} as select * from 
    read_csv_auto('{Path(file_path)}', parallel=True, all_varchar=1)
"""
duckdb.sql(sql_stmt)

# CatalogException: Catalog Error: Table with name "parking" already exists!

97.6 ms ± 483 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [29]:
tab_name = "parking"
df1 = duckdb.sql(f"select * from {tab_name} limit 5").df()

In [30]:
df1.columns 

Index(['Summons Number', 'Plate ID', 'Registration State', 'Plate Type',
       'Issue Date', 'Violation Code', 'Vehicle Body Type', 'Vehicle Make',
       'Issuing Agency', 'Street Code1', 'Street Code2', 'Street Code3',
       'Vehicle Expiration Date', 'Violation Location', 'Violation Precinct',
       'Issuer Precinct', 'Issuer Code', 'Issuer Command', 'Issuer Squad',
       'Violation Time', 'Time First Observed', 'Violation County',
       'Violation In Front Of Or Opposite', 'House Number', 'Street Name',
       'Intersecting Street', 'Date First Observed', 'Law Section',
       'Sub Division', 'Violation Legal Code', 'Days Parking In Effect',
       'From Hours In Effect', 'To Hours In Effect', 'Vehicle Color',
       'Unregistered Vehicle?', 'Vehicle Year', 'Meter Number',
       'Feet From Curb', 'Violation Post Code', 'Violation Description',
       'No Standing or Stopping Violation', 'Hydrant Violation',
       'Double Parking Violation', 'Latitude', 'Longitude', 'Communit

In [31]:
df1

,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,...,Hydrant Violation,Double Parking Violation,Latitude,Longitude,Community Board,Community Council,Census Tract,BIN,BBL,NTA
0,1283294138,GBB9093,NY,PAS,08/04/2013,46,SUBN,AUDI,P,37250,...,None,None,None,None,None,None,None,None,None,None
1,1283294151,62416MB,NY,COM,08/04/2013,46,VAN,FORD,P,37290,...,None,None,None,None,None,None,None,None,None,None
2,1283294163,78755JZ,NY,COM,08/05/2013,46,P-U,CHEVR,P,37030,...,None,None,None,None,None,None,None,None,None,None
3,1283294175,63009MA,NY,COM,08/05/2013,46,VAN,FORD,P,37270,...,None,None,None,None,None,None,None,None,None,None
4,1283294187,91648MC,NY,COM,08/08/2013,41,TRLR,GMC,P,37240,...,None,None,None,None,None,None,None,None,None,None


In [32]:
sql_stmt = f"""
    select count(*) from {tab_name}
"""
duckdb.sql(sql_stmt).df()

,count_star()
0,9100278


In [33]:
sql_stmt = f"""
    select "Registration State", count(*) from {tab_name}
    group by "Registration State"
    order by 2 desc, 1;
"""
duckdb.sql(sql_stmt).df()

,Registration State,count_star()
0,NY,7029804
1,NJ,878677
2,PA,225760
3,CT,136973
4,FL,111887
...,...,...
64,MB,22
65,YT,14
66,FO,9
67,NT,6
